The goal of this project is to explore if there are relationships between local economies and hospital quality.

The goal of this notebook is to explore if per capita personal income is related to hospital quality. Income data comes from https://apps.bea.gov/api/data/, and the hospital quality data comes from https://data.medicare.gov. The final product should be two SQL tables, "hospitals" and "incomes". There should be a complete many to one join between the "hospitals" and "incomes" tables. The joined table should result in there being economic data associated with each hospital

In [1]:
import pandas as pd
import sqlite3
import requests
import json
from sodapy import Socrata
import keys # file of API keys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import scipy as sp


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

ModuleNotFoundError: No module named 'keys'

## Create hospital dataframe

In [ ]:
# API call to the Data.Medicare.Gov Hospital General Information dataset
# The medicare data is available through Socrata Open Data API, which can be easily queried using sodapy
client = Socrata("data.medicare.gov", None)
# "xubh-q36u" is the hospital general information dataset id
results = client.get("xubh-q36u", limit = 10000)

# Convert to pandas DataFrame
hr_df = pd.DataFrame.from_records(results)
hr_df.describe()

# Because we are trying to map locations and values to hospitals we will focus on geography data. 
# The income data is done by county, we are going to use the county name and state data to map hospitals to 
# local economic metrics
# State is in 'state' using two letter abbr. and the county name is ALL CAPS in county_name. 

## Create per capita income dataframe

In [ ]:
# API Call to get average income per capita per county from SEA data
key = keys.sea_api_key
econ_addr = "https://apps.bea.gov/api/data/"

# 'CAINC1' is the personal income table, and linecode 3 is the per capita incomes
parameters = {
    "UserID" : key,
    "DATASETNAME" : 'Regional',
    "RESULTFORMAT" : 'JSON',
    "METHOD" : "GETDATA",
    'GEOFIPS': 'COUNTY',
    'TABLENAME': 'CAINC1',
    'YEAR': '2018',
    'LINECODE': '3'
}

response = requests.get(econ_addr, parameters)
inc_json = response.json()

# print(inc_json.keys()
# out: ['BEAAPI']
# print(inc_json['BEAAPI'].keys())
# out: ['Request', 'Results'])
# print(inc_json['BEAAPI']['Results'].keys()
# out: ['Statistic', 'UnitOfMeasure', 'PublicTable', 'UTCProductionTime', 'NoteRef', 'Dimensions', 'Data', 'Notes']

# once down to the flat data layer, then turn into a df
inc_data = inc_json['BEAAPI']['Results']['Data']
inc_df = pd.DataFrame(inc_data)
inc_df.describe()


# GeoFips = unique identifier for each region. Use as foreign key in hospitals table.
# GeoName is string of "county name, state", each is unique. These values will be what is mapped to 
# the hospital data to established which GeoFip should be added to the hospital table row.
# NoteREf: notes about the data. The notes have to do with how county definitons have changed
# over time, but not relevant given we are working with modern hospital data that should 
# be encodded using modern county definitions

In [ ]:
# Split the county and state name into seperate cols to be compared with the hospital

# using right split because the state comes after the last comma
county_state_split = inc_df.GeoName.str.rsplit(",", n=1, expand=True)
inc_df['county_name'] = county_state_split[0]
inc_df['state'] = county_state_split[1]


# need to make the county name all upper to match the econ data
inc_df['county_name'] = inc_df['county_name'].str.upper().str.strip()

# The hospital data lists the county of hospitals in independent cities as:
# X CITY (e.g. ALEXANDRIA CITY)
# where the SEA data lists as:
# "X (INDEPENDENT CITY)" (e.g. ALEXANDRIA (INDEPENDENT CITY))
# so convert to same string format
ind_city = (inc_df.county_name.str.contains("(INDEPENDENT CITY)", regex=False))
inc_df.loc[ind_city, 'county_name'] = inc_df.loc[ind_city, 'county_name'].str.replace(
    "(INDEPENDENT CITY)", "CITY", regex=False)

# Remove the potentially redundant city added in if the independent city's name is X CITY
# e.g. Carson City
city_city = (inc_df.county_name.str.contains("CITY CITY", regex=False))
inc_df.loc[city_city, 'county_name'] = inc_df.loc[city_city, 'county_name'].str.replace(
    "CITY ", "", n=1, regex=False)         
            
# Because there are some very small independent cities, they are often grouped with their
# nearby county for census purposes. The income data has the listed county for these places
# as "COUNTY + CITY" or "COUNTY, CITY + CITY" 
# So these need to be split out into individual cols, and for the every col
# After the first one, there needs to be a  "CITY" appended to match the 
# syntax the hospital data uses for independent cities
# As a result, the economic data table won't just have a single county name value, but actually 
# multiple county/city names in the counties_split_cols 
counties_split_out = inc_df.county_name.str.split(r" \+ |, ", expand = True)
counties_split_cols = ["county_name_{}".format(i) for i in range(counties_split_out.shape[1])]
inc_df[counties_split_cols] = counties_split_out
for col in counties_split_cols[1:]:
    inc_df[col] = inc_df[col] + " CITY"

# drop the county name col now that it has been split out
inc_df = inc_df.drop('county_name', 1)

# need to remove the "*" that imply there exists a note on the state abbr.
inc_df['state'] = inc_df['state'].str.strip().str.replace("*", "")

# Make the data values numeric. The income data uses commas in the numbers
# that need to be removed
inc_df['DataValue'] = pd.to_numeric(inc_df['DataValue'].str.replace(",", ""), errors = 'coerce')

# drop the 25 cols that have nans. These are county designations that 
# have been used in the past, but not in 2018
inc_df = inc_df.dropna(subset = ['DataValue'])

In [ ]:
# Manual modifications of the dataset

# The income dataset does not have income data for US territories, so drop out the hospital data
territories = ['VI', 'AS', 'MP', 'GU', 'PR']
hr_df = hr_df.loc[~hr_df.state.isin(territories)]

# Fixing typos in the hospital names.
# TODO: remove manual typo correction and use fuzzywuzzy
typos = {"NORTHWEST ARTIC BOROUGH" : "NORTHWEST ARCTIC BOROUGH", "NORTH SLOPE BOROUH" : "NORTH SLOPE BOROUGH",
         'JEFFRSON DAVIS' : 'JEFFERSON DAVIS','E. BATON ROUGE' : 'EAST BATON ROUGE', 'ST. JOHN BAPTIST': 
         'ST. JOHN THE BAPTIST', 'SITKA BOROUGH' : 'SITKA CITY AND BOROUGH', 'DONA ANA': 'DOÑA ANA',
         'LAKE OF  WOODS' : 'LAKE OF THE WOODS', 'YELLOW MEDCINE': 'YELLOW MEDICINE', 'SCOTT BLUFF': 'SCOTTS BLUFF',
         'NORTHUMBERLND' : 'NORTHUMBERLAND', 'LAPAZ': 'LA PAZ'}
for key in typos:
    hr_df.loc[hr_df.county_name == key, "county_name"] = typos[key]

In [ ]:
# The first step is to find the exact matches between
# the state and county_name from the hospital df and the state and first counties_split_cols in the income df
# df is going to be the final hosptial df that we turn into a SQL table. The goal is it to be the same as
# the original table with a new col named GeoFips that corresponds to the correct GeoFip in the income 
# data table

# By doing an inner merge on, only going to get rows where both match togehter. By only selecting for hr_df cols
# and "GeoFips", df is just the hr_df where there is an exact match and the correct GeoFip for the hospital
df = pd.merge(hr_df, inc_df,  how='inner', left_on=['county_name','state'],
              right_on=[counties_split_cols[0],'state'], suffixes = [None, None])[list(hr_df) + ['GeoFips']]


# Remove the rows from hr_df that are now in df
hr_df = hr_df.loc[~hr_df.provider_id.isin(df.provider_id)]
print(hr_df.shape)
print(df.shape)
# We have perfect matches for all but 102 rows

In [ ]:
# need to check for matches on other counties_split_cols. Could do with inner merge like for the first call,
# but this is an alternate approach.
# First, left join the remaining unmatched rows of the hr_dr and the inc_df on state only
alt_matches_df = pd.merge(hr_df, inc_df,  how='left', on='state')
# Then find rows where hr_df matches any of the counties_split_cols
alt_matches_df = alt_matches_df.loc[alt_matches_df[counties_split_cols].isin(alt_matches_df.county_name).any(1)]
# Take those rows and append to the df with only the columns from the df
df = pd.concat([df, alt_matches_df[list(df)]])
hr_df = hr_df.loc[~hr_df.provider_id.isin(df.provider_id)]
# Update the hr_df with the missing
print(alt_matches_df.shape)
print(hr_df.shape)
print(df.shape)
# we added 19 more rows to the df, and have 83 unmatched

In [ ]:
# DC just uses a different name for the county name between the two datasets, but both consider it
# a single county, so just join by state only
# Get the DC GeoFip
dc_ind = inc_df.loc[inc_df.state == "DC", 'GeoFips'].values[0]
hr_df.loc[hr_df.state == "DC"]
# Assign to the hr_df DC rows
temp = hr_df.loc[hr_df.state == "DC"]
# append new rows to df and remove from hr_df
df = pd.concat([df, temp])
df.loc[df.state == "DC", "GeoFips"] = dc_ind
hr_df = hr_df.loc[~hr_df.provider_id.isin(df.provider_id)]
print(temp.shape)
print(hr_df.shape)
print(df.shape)
# we added 9 more rows to the df, and have 74 unmatched

In [ ]:
# Do the exact same thing as the original merge, but remove spaces and apostrophes from both county_name
# cols to enable an inexact match
# TODO: Do this step with fuzzywuzzy

hr_df['county_name_ns'] = hr_df.county_name.str.replace(" ", "", regex = False).str.replace("\'", "", regex = False)
inc_df[counties_split_cols[0] + "_na"] = inc_df[counties_split_cols[0]].str.replace(" ", "", regex = False).str.replace("\'", "", regex = False)
temp = pd.merge(hr_df, inc_df,  how='inner', left_on=['county_name_ns','state'],
              right_on=[counties_split_cols[0]  + "_na",'state'], suffixes = [None, None])[list(df)]

# remove the hr_df['county_name_ns'] to keep the columns the same as df before appending
temp = temp.drop('county_name_ns', 1)
# append new rows to df and remove from hr_df
df = pd.concat([df, temp])
hr_df = hr_df.loc[~hr_df.provider_id.isin(df.provider_id)]
print(temp.shape)
print(hr_df.shape)
print(df.shape)
# we added 55 more rows to the df, and have 19 unmatched

In [ ]:
# Alaskan county names are written out more formally in the inc_data than in the hr_df
# So do a substring search to see if the shorter name from hr_df is in the longer name 
# of the inc_df
# TODO: Do this step with fuzzywuzzy

# Do a left merge on "state" for Alaska only. This creates a many to many match for each row in the hr_dr
# From there, can check if the hr_dr county name is a substring of the counties_split_cols[0]. Keep
# only the rows where it is true
temp = pd.merge(hr_df.loc[hr_df.state == "AK"], inc_df,  how='left', on='state')
# use a lambda function to see if col is substring of other col
temp = temp.loc[temp.apply(lambda x: x.county_name in x[counties_split_cols[0]], axis=1)]

# append new rows to df and remove from hr_df
df = pd.concat([df, temp[list(df)]])
hr_df = hr_df.loc[~hr_df.provider_id.isin(df.provider_id)]
print(temp.shape)
print(hr_df.shape)
print(df.shape)
# we added 19 more rows to the df, and have 0 unmatched

In [ ]:
# create a dummy sql server in memory
conn = sqlite3.connect(":memory:")

In [ ]:
# need to convert the geocode col which is a dictionary to a string for sql encoding
df['geocoded_column'] = df['geocoded_column'].astype(str)
df.to_sql('hospitals', con=conn)

# 'SELECT FirstName, LastName    -- list out: first names and surnames
#   FROM Patients
#   WHERE NOT EXISTS (    -- patients who do not meet criteria
#     SELECT * FROM Appointments -- get all columns from selection
#         WHERE Patients.PatientID = Appointments.PatientID -- appointment is for the patient
#           AND Appointments.AppointmentDate > DATEADD(year, -1, GETDATE())    -- appointments in last year
#           AND Appointments.AppointmentType = 1)    -- appointments that are AWVs'

In [ ]:
inc_df.to_sql('income', con=conn)

In [ ]:
# left join of hospitals and income on GeoFips filtered for hospitals
# where the average income is greater than 50k 
query = """SELECT * 
           FROM hospitals 
           LEFT JOIN income ON hospitals.GeoFips = income.GeoFips
           WHERE DATAVALUE > 75000"""

n_df = pd.read_sql_query(query, conn)

In [ ]:
query = """SELECT * 
           FROM hospitals 
           LEFT JOIN income ON hospitals.GeoFips = income.GeoFips"""

n_df = pd.read_sql_query(query, conn)
n_df['hospital_overall_rating'] = pd.to_numeric(n_df['hospital_overall_rating'], errors= 'coerce')

n_df = n_df.dropna(subset = ['DataValue', 'hospital_overall_rating'])
sns.boxplot(y = 'DataValue', x = 'hospital_overall_rating', data = n_df)
plt.figure()
sns.distplot(n_df.DataValue)
plt.figure()
sns.distplot(n_df.hospital_overall_rating)

#There is a weak, but significant correlation between per capita income and overall hospital quality
sp.stats.spearmanr(n_df.DataValue, n_df.hospital_overall_rating)